# Multispectral Imagery Visualizations (Beginner Lab)

## Libraries and datasets (quick reference)
1. This notebook uses Earth Engine and geemap.
2. Dataset: Sentinel-2 surface reflectance (`COPERNICUS/S2_SR_HARMONIZED`).

![Placeholder: Three panels labeled True Color (RGB), False Color (IRG), and a Lake AOI outline.]()

## What you will do
1. Load a Lake Lagunita area-of-interest (AOI) from a GeoJSON file.
2. Build a median composite from the last month of Sentinel-2 imagery.
3. Visualize RGB and IRG composites with dynamic contrast stretching.


In [ ]:
# Install required libraries. This works in both Colab and local Jupyter.
!pip install -q --upgrade earthengine-api geemap


In [ ]:
# Import Earth Engine so we can access datasets and run geospatial analysis.
import ee  # Earth Engine Python API

# Import geemap for interactive maps inside notebooks.
import geemap  # Map display helper for Earth Engine

project='YOUR PROJECT ID HERE'

# Try to initialize Earth Engine. If it fails, run authentication first.
# This works in both Colab and local Jupyter.
try:
    ee.Initialize(project=project)  # Connect to Earth Engine using saved credentials
except Exception:
    ee.Authenticate()  # Open a browser-based login flow
    ee.Initialize(project=project)  # Retry initialization after authentication

# Create a map so we can visualize results.
Map = geemap.Map()  # Interactive map widget

# Confirm that Earth Engine initialized successfully.
print('Earth Engine initialized.')

In [ ]:
# -----------------------------------------------------------------------------
# FOUNDATIONAL CONCEPT: Area of Interest (AOI)
# An AOI is the geographic boundary we want to study. We will load a GeoJSON
# file and convert it into an Earth Engine geometry.
# -----------------------------------------------------------------------------

import json  # Read GeoJSON files
from pathlib import Path  # Build file paths safely

# Try common locations for the GeoJSON file (repo root or notebooks folder).
possible_paths = [
    Path('notebooks/output/lakelagunita.geojson'),
    Path('output/lakelagunita.geojson')
]

geojson_path = None  # Placeholder for the path we find
for candidate in possible_paths:
    if candidate.exists():
        geojson_path = candidate
        break

# Stop if we cannot find the file.
if geojson_path is None:
    raise FileNotFoundError('Could not find lakelagunita.geojson in notebooks/output or output.')

# Load the GeoJSON file.
with geojson_path.open('r', encoding='utf-8') as f:
    geojson_data = json.load(f)  # Read JSON into a Python dictionary

# Convert the GeoJSON into an Earth Engine object.
# geemap.geojson_to_ee returns an ee.FeatureCollection.
aoi_fc = geemap.geojson_to_ee(geojson_data)  # FeatureCollection

aoi = aoi_fc.geometry()  # Extract geometry for filtering

# Add AOI outline to the map.
Map.addLayer(aoi, {'color': 'yellow'}, 'Lake Lagunita AOI')
Map.centerObject(aoi, 14)  # Zoom to the AOI
Map


In [ ]:
# -----------------------------------------------------------------------------
# FOUNDATIONAL CONCEPT: Median composite from the last month
# A median composite reduces clouds and noise by taking the median pixel value
# across many images in a time window.
# -----------------------------------------------------------------------------

from datetime import datetime, timedelta  # Work with dates

# Define the last 30 days as a date range.
end_date = datetime.utcnow().date()  # Today's date (UTC)
start_date = end_date - timedelta(days=30)  # 30 days ago

# Convert dates to strings that Earth Engine understands.
start_str = start_date.strftime('%Y-%m-%d')
end_str = end_date.strftime('%Y-%m-%d')

# Load Sentinel-2 surface reflectance images.
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')  # Sentinel-2 SR
    .filterBounds(aoi)  # Keep images that overlap the AOI
    .filterDate(start_str, end_str)  # Last 30 days
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))  # Basic cloud filter
)

# -----------------------------------------------------------------------------
# FOUNDATIONAL CONCEPT: Cloud masking (Sentinel-2 SCL band)
# The SCL band classifies pixels (clouds, shadows, vegetation, water, etc.).
# We keep only classes that are usually clear or usable.
# -----------------------------------------------------------------------------

def mask_s2_sr(image):
    # Select the Scene Classification Layer (SCL).
    scl = image.select('SCL')

    # Keep these classes:
    # 4 = Vegetation, 5 = Bare soils, 6 = Water, 7 = Unclassified, 11 = Snow/Ice
    # This is a simple mask intended for beginners.
    mask = (
        scl.eq(4)
        .Or(scl.eq(5))
        .Or(scl.eq(6))
        .Or(scl.eq(7))
        .Or(scl.eq(11))
    )

    # Apply the mask and return the cleaned image.
    return image.updateMask(mask)

# Apply the cloud mask and create a median composite.
composite = collection.map(mask_s2_sr).median()

# Show the number of images used in the composite.
count = collection.size().getInfo()  # Bring count to client
print(f'Images in composite: {count}')


In [ ]:
# -----------------------------------------------------------------------------
# FOUNDATIONAL CONCEPT: Dynamic contrast stretching
# We compute percentiles in the AOI so the RGB/IRG display uses a data-driven
# min and max, improving visibility across different dates and lighting.
# -----------------------------------------------------------------------------

# Adjustable stretch percentage (2 means 2% clipped on each tail).
stretch_percent = 0  # Try 1, 2, or 5 for different contrast

# Helper function to compute dynamic min/max for a band set.
def get_dynamic_vis(image, bands, region, scale=10, stretch=2):
    # Reduce to percentiles for each band.
    stats = image.select(bands).reduceRegion(
        reducer=ee.Reducer.percentile([stretch, 100 - stretch]),
        geometry=region,
        scale=scale,
        maxPixels=1e9
    ).getInfo()

    # Collect per-band min and max values.
    mins = []
    maxs = []
    for b in bands:
        low_key = f'{b}_p{stretch}'
        high_key = f'{b}_p{100 - stretch}'
        if stats.get(low_key) is not None:
            mins.append(stats[low_key])
        if stats.get(high_key) is not None:
            maxs.append(stats[high_key])

    # Fallback if stats are missing.
    if not mins or not maxs:
        return {'min': 0, 'max': 3000, 'bands': bands}

    # Use the global min and max across the band set.
    return {
        'min': min(mins),
        'max': max(maxs),
        'bands': bands
    }


In [ ]:
# Create RGB (true color) and IRG (false color) composites.
# Sentinel-2 band meanings:
# B4 = Red, B3 = Green, B2 = Blue, B8 = Near Infrared (NIR)

# RGB (true color)
rgb_bands = ['B4', 'B3', 'B2']
rgb_vis = get_dynamic_vis(composite, rgb_bands, aoi, scale=10, stretch=stretch_percent)

# IRG (false color: NIR, Red, Green)
irg_bands = ['B8', 'B4', 'B3']
irg_vis = get_dynamic_vis(composite, irg_bands, aoi, scale=10, stretch=stretch_percent)

# Add layers to the map.
Map.addLayer(composite, rgb_vis, 'Sentinel-2 RGB (Dynamic Stretch)')
Map.addLayer(composite, irg_vis, 'Sentinel-2 IRG (Dynamic Stretch)')

# Display the map.
Map


## Try it

1. Change the `stretch_percent` value to see how contrast changes.
2. Increase the date range to 60 or 90 days and compare the composite.
3. Zoom out and explore how water and vegetation look in the IRG view.


In [ ]:
# Try changing these values and re-running the composite and visualization cells.
# Longer date ranges may reduce noise but can blur short-term changes.

# Example adjustments:
# stretch_percent = 5
# start_date = end_date - timedelta(days=60)
